## Import

In [17]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

Dataset Info.

##### train.csv [파일]  
PRODUCT_ID : 제품의 고유 ID  
Y_Class : 제품 품질 상태(Target)   
0 : 적정 기준 미달 (부적합)  
1 : 적합  
2 : 적정 기준 초과 (부적합)  
Y_Quality : 제품 품질 관련 정량적 수치  
TIMESTAMP : 제품이 공정에 들어간 시각  
LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)  
PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)  
X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수  
  
  
##### test.csv [파일]  
PRODUCT_ID : 제품의 고유 ID  
TIMESTAMP : 제품이 공정에 들어간 시각  
LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)  
PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)  
X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수  
  
  
sample_submission.csv [파일] - 제출 양식  
PRODUCT_ID : 제품의 고유 ID  
Y_Class : 예측한 제품 품질 상태  
0 : 적정 기준 미달 (부적합)  
1 : 적합  
2 : 적정 기준 초과 (부적합)  
  
실제 공정 과정에서의 데이터로, 보안상의 이유로 일부 변수가 비식별화 처리 되었습니다. (X변수)  
'LINE', 'PRODUCT_CODE'는 Train / Test 모두 동일한 종류가 존재합니다.  

## Data Load

In [ ]:
# make [] 

train_col_year = []
train_col_month = []
train_col_day = []

test_col_year = []
test_col_month = []
test_col_day = []


# split : year, month, day

for i in range(len(train_time_df)):
    time_df_i = train_time_df[i]    
    year = time_df_i[:4]
    train_col_year.append(year)

    month = time_df_i[5:7]
    train_col_month.append(month)

    day = time_df_i[8:10]
    train_col_day.append(day)

for i in range(len(test_time_df)):
    time_df_i = test_time_df[i]    
    year = time_df_i[:4]
    test_col_year.append(year)

    month = time_df_i[5:7]
    test_col_month.append(month)

    day = time_df_i[8:10]
    test_col_day.append(day)


# 
train_df['year'] = train_col_year
train_df['month'] = train_col_month
train_df['day'] = train_col_day

test_df['year'] = test_col_year
test_df['month'] = test_col_month
test_df['day'] = test_col_day

# astype
train_df = train_df.astype({'year':'int'})
train_df = train_df.astype({'month':'int'})
train_df = train_df.astype({'day':'int'})

test_df = test_df.astype({'year':'int'})
test_df = test_df.astype({'month':'int'})
test_df = test_df.astype({'day':'int'})

In [3]:
import pandas as pd
import numpy as np


train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_time_df = train_df['TIMESTAMP']
test_time_df = test_df['TIMESTAMP']

def split_TIMESTEPS(train_df, test_df, train_time_df, test_time_df):

    # make [] 

    train_col_year = []
    train_col_month = []
    train_col_day = []

    test_col_year = []
    test_col_month = []
    test_col_day = []


    # split : year, month, day

    for i in range(len(train_time_df)):
        time_df_i = train_time_df[i]    
        year = time_df_i[:4]
        train_col_year.append(year)

        month = time_df_i[5:7]
        train_col_month.append(month)

        day = time_df_i[8:10]
        train_col_day.append(day)
    
    for i in range(len(test_time_df)):
        time_df_i = test_time_df[i]    
        year = time_df_i[:4]
        test_col_year.append(year)

        month = time_df_i[5:7]
        test_col_month.append(month)

        day = time_df_i[8:10]
        test_col_day.append(day)


    # 
    train_df['year'] = train_col_year
    train_df['month'] = train_col_month
    train_df['day'] = train_col_day

    test_df['year'] = test_col_year
    test_df['month'] = test_col_month
    test_df['day'] = test_col_day

split_TIMESTEPS(train_df, test_df, train_time_df, test_time_df)

In [4]:
# astype

train_df = train_df.astype({'year':'int'})
train_df = train_df.astype({'month':'int'})
train_df = train_df.astype({'day':'int'})

test_df = test_df.astype({'year':'int'})
test_df = test_df.astype({'month':'int'})
test_df = test_df.astype({'day':'int'})

In [5]:
train_df.describe()

,Y_Class,Y_Quality,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
count,598.000000,598.000000,349.000000,349.000000,349.0,349.0,349.000000,349.0,349.000000,349.000000,...,100.000000,100.00000,100.000000,100.000000,100.00000,99.0,0.0,0.0,0.0,0.0
mean,1.025084,0.530896,2.409742,95.123209,0.0,45.0,10.392550,0.0,48.802292,10.048711,...,50.807300,53.60770,49.606200,51.659800,66.64970,1.0,NaN,NaN,NaN,NaN
std,0.565069,0.007401,5.895256,4.107640,0.0,0.0,0.489019,0.0,4.373824,0.215571,...,7.011828,8.13899,7.158917,8.913065,4.52781,0.0,NaN,NaN,NaN,NaN
min,0.000000,0.500856,1.000000,87.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,32.120000,31.70000,32.560000,30.490000,61.67000,1.0,NaN,NaN,NaN,NaN
25%,1.000000,0.527535,2.000000,93.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,49.485000,52.20000,42.160000,49.915000,63.64500,1.0,NaN,NaN,NaN,NaN
50%,1.000000,0.530436,2.000000,95.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,53.425000,55.92500,51.460000,56.175000,65.14000,1.0,NaN,NaN,NaN,NaN
75%,1.000000,0.533433,2.000000,98.000000,0.0,45.0,11.000000,0.0,51.000000,10.000000,...,55.287500,58.97500,55.030000,57.175000,67.11500,1.0,NaN,NaN,NaN,NaN
max,2.000000,0.578841,103.000000,102.000000,0.0,45.0,11.000000,0.0,62.000000,11.000000,...,60.240000,68.66000,60.410000,59.930000,79.75000,1.0,NaN,NaN,NaN,NaN


In [5]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

## Data Pre-processing

In [6]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [7]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


## Split train / valid

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, random_state=42, test_size=0.2)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(478, 2880)
(478,)
(120, 2880)
(120,)


## Classification Model Fit

#### Train / Valid

In [58]:
### 나중에 optuna 적용해보기 ###


from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

def model_fit_pred(x_train, y_train, x_val, y_val):
    
    ### RandomForestClassifier ###
    RF = RandomForestClassifier(random_state=42).fit(x_train, y_train)
    print('RandomForestClassifier >>>>> Done', '\n\n')
    rf_pred = RF.predict(x_val)
    print(classification_report(y_val, rf_pred))

    ### XGBClassifier ###
    XGB = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.01, objective='multi:softmax').fit(x_train, y_train)
    print('\n\n', 'XGBClassifier >>>>> Done', '\n\n')
    xgb_pred = XGB.predict(x_val)
    print(classification_report(y_val, xgb_pred))   

    submit = pd.read_csv('./sample_submission.csv')
    submit['Y_Class'] = xgb_pred
    submit.to_csv('./xgb_submission.csv', index=False)



In [59]:
model_fit_pred(x_train, y_train, x_val, y_val)

RandomForestClassifier >>>>> Done 


              precision    recall  f1-score   support

           0       0.78      0.30      0.44        23
           1       0.72      0.96      0.82        76
           2       0.78      0.33      0.47        21

    accuracy                           0.73       120
   macro avg       0.76      0.53      0.57       120
weighted avg       0.74      0.72      0.68       120

[00:00:21] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 XGBClassifier >>>>> Done 


              precision    recall  f1-score   support

           0       0.86      0.26      0.40        23
           1       0.70      0.96      0.81        76
           2       0.78      0.33  

(array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1]),
 array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2,
        1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1]))

In [11]:
### RF ###

RF = RandomForestClassifier(random_state=42).fit(x_train, y_train)
print('Done.')

from sklearn.metrics import classification_report

pred = RF.predict(x_val)
print(classification_report(y_val, pred))

Done.
              precision    recall  f1-score   support

           0       0.78      0.30      0.44        23
           1       0.72      0.96      0.82        76
           2       0.78      0.33      0.47        21

    accuracy                           0.73       120
   macro avg       0.76      0.53      0.57       120
weighted avg       0.74      0.72      0.68       120



In [ ]:
# xgb (1)

import xgboost as xgb

dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)

# specify parameters via map
param = {"max_depth": 100, "eta": 1, "objective": "multi:softmax"}

# specify validations set to watch performance
watchlist = [(dval, "eval"), (dtrain, "train")]

# number of boosting rounds
num_round = 2
bst = xgb.train(param, dtrain, num_boost_round=num_round, evals=watchlist)
print('XGBClassifier >>>>> Done', '\n\n')

# run prediction
preds = bst.predict(dval)
labels = dval.get_label()


In [61]:
# xgb (2)

XGB = XGBClassifier(n_estimators=1, max_depth=2, learning_rate=0.01, objective='multi:softmax').fit(x_train, y_train)
print('\n\n', 'XGBClassifier >>>>> Done', '\n\n')
xgb_pred = XGB.predict(x_val)
print(classification_report(y_val, xgb_pred))  

# submit = pd.read_csv('./sample_submission.csv')
# submit['Y_Class'] = xgb_pred
# submit.to_csv('./xgb_submission.csv', index=False)

[00:06:35] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 XGBClassifier >>>>> Done 


              precision    recall  f1-score   support

           0       0.88      0.30      0.45        23
           1       0.73      0.92      0.81        76
           2       0.56      0.43      0.49        21

    accuracy                           0.72       120
   macro avg       0.72      0.55      0.58       120
weighted avg       0.73      0.72      0.69       120



ValueError: Length of values (120) does not match length of index (310)

#### Train / Test

In [62]:
XGB = XGBClassifier(n_estimators=500, max_depth=2, learning_rate=0.01, objective='multi:softmax').fit(train_x, train_y)
print('\n\n', 'XGBClassifier >>>>> Done', '\n\n')
xgb_pred = XGB.predict(test_x)

[00:10:15] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 XGBClassifier >>>>> Done 




In [63]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = xgb_pred
submit.to_csv('./xgb_submission.csv', index=False)

## Inference

In [ ]:
preds = RF.predict(test_x)
print('Done.')

Done.


## Submit

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

submit['Y_Class'] = preds

submit.to_csv('./baseline_submission.csv', index=False)